# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
print('working directory: ' + os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# checking your folder and subfolder event data directory
print('folder and subfolder event data: ' + filepath)

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

working directory: d:\WORKING AREA\DATA ENGINEER\workplace-github\project2-data-modeling-with-cassandra
folder and subfolder event data: d:\WORKING AREA\DATA ENGINEER\workplace-github\project2-data-modeling-with-cassandra/event_data
['d:\\WORKING AREA\\DATA ENGINEER\\workplace-github\\project2-data-modeling-with-cassandra/event_data\\2018-11-01-events.csv', 'd:\\WORKING AREA\\DATA ENGINEER\\workplace-github\\project2-data-modeling-with-cassandra/event_data\\2018-11-02-events.csv', 'd:\\WORKING AREA\\DATA ENGINEER\\workplace-github\\project2-data-modeling-with-cassandra/event_data\\2018-11-03-events.csv', 'd:\\WORKING AREA\\DATA ENGINEER\\workplace-github\\project2-data-modeling-with-cassandra/event_data\\2018-11-04-events.csv', 'd:\\WORKING AREA\\DATA ENGINEER\\workplace-github\\project2-data-modeling-with-cassandra/event_data\\2018-11-05-events.csv', 'd:\\WORKING AREA\\DATA ENGINEER\\workplace-github\\project2-data-modeling-with-cassandra/event_data\\2018-11-06-events.csv', 'd:\\WORKI

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# check if whether event_datafile_new.csv is existing or not then remove
if os.path.exists('event_datafile_new.csv'):
    os.remove('event_datafile_new.csv')
    print('Remove existing event_datafile_new.csv file')
    
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
print('Wrote all records into new full event csv file')


Remove existing event_datafile_new.csv file
Wrote all records into new full event csv file


In [22]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [24]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## Drop the tables before create table

In [ ]:
# Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS song_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query2 = "DROP TABLE IF EXISTS artist_table"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "DROP TABLE IF EXISTS user_table"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




Query 1:
=> **song_table:** filtering by session_id and item_in_session. So both fields are part of the partition key.

| Column          | Type  | Partition Key | Cluster Key | Explanation                 |
| --------------- | ----- | ------------- | ----------- | --------------------------- |
| session_id      | INT   | Y             |             | fitering by session_id      |
| item_in_session | INT   | Y             |             | fitering by item_in_session |
| artist          | TEXT  |               |             |                             |
| song_title      | TEXT  |               |             |                             |
| length          | FLOAT |               |             |                             |

In [26]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS song_table"
query = query + "(session_id int, item_in_session int, artist text, song_title text, length float, \
                PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [27]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert records into song_table
        query = "INSERT INTO song_table (session_id, item_in_session, artist, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))  #Refer: https://knowledge.udacity.com/questions/529901

#### Do a SELECT **QUERY** 1 to verify that the data have been inserted into each table

In [28]:
query = "SELECT artist, song_title, length \
FROM song_table WHERE session_id=338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Query 2:
=> **artist_table:** 
- Filtering by user_id and session_id. So both fields are composite partition key.
- sorted by itemInSession => item_in_session is cluster key
  
| Column          | Type | Partition Key | Cluster Key | Explanation                                     |
| --------------- | ---- | ------------- | ----------- | ------------------------------------------------ |
| user_id         | INT  | Y             |             | fitering by userId                               |
| session_id      | INT  | Y             |             | fitering by session_id                           |
| artist          | TEXT |               |             |                                                  |
| song_title      | TEXT |               |             |                                                  |
| item_in_session | INT  |               | Y           | sorted by itemInSession => item_in_session is CK |
| first_name      | TEXT |               |             |                                                  |
| last_name       | TEXT |               |             |                                                  |

In [29]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 

query = "CREATE TABLE IF NOT EXISTS artist_table"
query = query + "(user_id int, session_id int, artist text, song_title text, item_in_session int, \
                first_name text, last_name text, \
                PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [30]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert records into artist_table
        query = "INSERT INTO artist_table (user_id, session_id, artist, song_title, item_in_session, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))  #Refer: https://knowledge.udacity.com/questions/529901

#### Do a SELECT **QUERY** 2 to verify that the data have been inserted into each table

In [31]:
query = "SELECT artist, song_title, item_in_session, first_name, last_name \
FROM artist_table WHERE user_id=10 AND session_id=182 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.item_in_session, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Query 3:
=> **user_table:** 
- Filtering by song_title . So that field is partition key.
- song_title is not unique => add user_id is CK

| Column     | Type | Partition Key | Cluster Key | Explanation                                   |
| ---------- | ---- | ------------- | ----------- | ---------------------------------------------- |
| song_title | TEXT | Y             |             | fitering by song => song_title is PK           |
| user_id    | INT  |               | Y           | due to song is not unique => add user_id is CK |
| first_name | TEXT |               |             |                                                |
| last_name  | TEXT |               |             |                                                |

In [32]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS user_table"
query = query + "(song_title text, user_id int, first_name text, last_name text, \
                PRIMARY KEY ((song_title), user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [33]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert records into user_table
        query = "INSERT INTO user_table (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))  #Refer: https://knowledge.udacity.com/questions/529901

#### Do a SELECT **QUERY** 3 to verify that the data have been inserted into each table

In [34]:
query = "SELECT first_name, last_name \
FROM user_table WHERE song_title='All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


## Drop the tables before closing out the sessions

### Drop the table before closing out the sessions

In [35]:
# Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS song_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query2 = "DROP TABLE IF EXISTS artist_table"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "DROP TABLE IF EXISTS user_table"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()